# Un modèle de langage

Qu’il soit sobre (*Small Language Model*), volumineux (*Large Language Model*) ou prévu pour un domaine particulier (*Specialized Language Model*), un modèle de langage est une représentation statistique de la distribution des symboles d’une langue (lettres, phonèmes, mots…) en vue d’effectuer des prédictions. En se basant par exemple sur la fréquence d’apparition des lettres dans un corpus, il est tout à fait envisageable de générer du texte. Ou plutôt une collection de lettres. Des outils supplémentaires peuvent être intégrés au modèle, comme un lexique, ou une distribution de la longueur des mots et des phrases dans un texte.

La limite principale d’un modèle de langage est qu’il est incapable de rendre compte d’une langue, tout au plus de son état à un certain moment donné en vue des paramètres initiaux qui lui ont été fournis. Il repose fondamentalement sur un corpus, qui lui-même est une extraction, et reproduira ses biais. Si votre corpus contient *La montagne de l’âme* de Gao Xingjian, il risque de donner trop d’importance aux formes conjuguées des verbes à la deuxième personne du singulier. Et si vous incluez l’ensemble du contenu d’Internet, qui lui-même est de plus en plus alimenté par des IA génératives, vous ne manquerez pas de véhiculer ses fictions.

## Aperçu de la difficulté de la tâche

Compter des occurrences revient à subdiviser un objet en éléments et à compter le nombre de fois où chaque élément apparaît. Prenons une liste aléatoire de cent nombres entiers entre 0 et 20 et calculons combien de fois apparaît le nombre 9 :

In [ ]:
from random import randint

l = [ randint(0, 20) for n in range(0, 100) ]

print(f"Le nombre 9 apparaît { l.count(9) } fois.")

On peut réaliser une opération similaire en comptabilisant les apparitions de la lettre *e* dans une phrase :

In [ ]:
sent = "En pratique, un pêcheur pêche avec une canne."

print(f"La lettre 'e' apparaît { sent.count('e') } fois.")

Python dissocie par défaut le caractère *e* de ses versions accentuée *ê* et majuscule *E*. Plusieurs stratégies peuvent être mises en place pour y remédier :

- remplacer les caractères ;
- additionner les occurrences de chaque cas ;
- normaliser la phrase.

Prenons la dernière stratégie en faisant appel à la méthode `.normalize()` du module `unicodedata`, qui décompose une lettre en ses différents constituants :

In [ ]:
import unicodedata

def normalize(s):
    """Returns the normalized version of a string.

    s -- string to normalize
    """
    normalized_string = str()
    for c in s:
        components = unicodedata.normalize('NFKD', c)
        base = components[0]
        normalized_string += base.lower()

    return normalized_string

print(normalize(sent))

Grâce à la fonction `map()`, on peut obtenir le même résultat plus rapidement :

In [ ]:
sent = map(lambda x: unicodedata.normalize('NFKD', x)[0], sent)
sent = ''.join(sent)

print(f"La lettre 'e' apparaît { sent.count('e') } fois.")

Et pour basculer en bas de casse :

In [ ]:
sent = sent.lower()

print(f"La lettre 'e' apparaît { sent.count('e') } fois.")

## Traduire le langage en probabilités

Quelle est la probabilité de tirer la lettre *A* au *Scrabble* ? La question, anodine, ne peut se résoudre que si nous connaissons deux quantités :

- le nombre total de jetons ;
- le nombre de jetons *A*.

### Calculer des fréquences absolues

Le calcul de fréquences se trouve ainsi au cœur des modèles probabilistes et constitue souvent la première mesure statistique d’un corpus. Considérons la phrase suivante :

> Le petit chat boit du lait.

Et faisons l’inventaire des caractères :

|Lettre|Fréquence|Cumul|
|:-:|:-:|:-:|
|t|5|5|
||5|10|
|i|3|13|
|a|2|15|
|e|2|17|
|l|2|19|
|b|1|20|
|c|1|21|
|d|1|22|
|h|1|23|
|o|1|24|
|p|1|25|
|u|1|26|
|.|1|27|

Réaliser la tâche en Python passe par l’objet `Counter` :

In [ ]:
from collections import Counter

sent = "Le petit chat boit du lait."

unigrams = Counter(sent.lower())

Le résultat étant hérité du type `dict`, on peut accéder à la fréquence d’un caractère particulier en interrogeant la clé ou en utilisant la méthode `.get()` qui a l’avantage de ne pas lever d’exception si une clé n’est pas présente :

In [ ]:
print(
    unigrams['t'],
    unigrams.get('e'),
    unigrams.get('j'),
    unigrams.get('z', 0),
    sep="\n"
)

Une autre méthode utile permet d’un coup d’œil d’afficher la liste des items les plus fréquents :

In [ ]:
unigrams.most_common(3)

Et pour connaître le nombre total d’occurrences ? La méthode `.values()` d’un objet `dict` permettant d’exposer la liste de toutes ses valeurs, nous pouvons ensuite facilement en établir la somme :

In [ ]:
N = sum(unigrams.values())

print(N)

### La fréquence relative

Le dénombrement des mots dans un texte ne donne qu’une mesure absolue de la présence de chacun, sans rien dire de leur importance. Qu’un terme apparaisse trois mille fois est en soi beaucoup, mais au milieu d’un corpus de trois milliards de mots, il ne pèse guère. D’où la nécessité de toujours considérer un chiffre parmi son environnement.

Calculer la fréquence relative d’une lettre revient à diviser le nombre de fois où elle apparaît avec la taille du corpus. En probabilités, la formule revient à :

$$
P(c) = \frac{F(c)}{N}
$$

Dans la formule, $P(c)$ est la probabilité de réalisation de l’événement $c$ – à savoir un caractère, $F(c)$ la fréquence d’apparition du caractère et $N$ la taille du corpus. En l’appliquant à la phrase plus haut, on peut en déduire que la probabilité de l’événement *i* est égale à :

$$
P(i) = \frac{3}{27} \approx 0.11
$$

Avec Python :

In [ ]:
p_i = unigrams.get('i') / N

Dans ce contexte, la réalisation d’un événement aléatoire est jugée indépendante des autres événements. Aussi, la probabilité de réaliser l’événement *i* puis l’événement *t* revient à multiplier les probabilités des deux événements :

$$
P(i \cap t) = P(i) \times P(t) = 0.11 \times 0.18 \approx 0.0206
$$

In [ ]:
p_i * unigrams.get('t') / N

### Modèle *n*-grammes

Quelle serait maintenant la probabilité que l’événement *it* se produise ? La question est différente de la précédente comme on considère l’ensemble *it* comme un événement et plus comme deux événements distincts. Il est par conséquent possible d’appliquer la première formule :

$$
P(it) = \frac{F(it)}{N-1}
$$

Pourquoi $N-1$ ? Les fréquences calculées précédemment n’impliquaient que des unigrammes (ou 1-grammes) or, ici, nous souhaitons estimer la probabilité d’un bigramme (ou 2-grammes). Caclulons le nombre de bigrammes dans la phrase :

In [ ]:
bigrams = Counter([
    sent[i:i+2]
    for i in range(len(sent) - 1)
])
sum(bigrams.values())

La règle est linéaire, de telle manière qu’il existe $N-2$ trigrammes, $N-3$ tétragrammes, $N-4$ 5-grammes etc. La probabilité de l’événement *it* s’établit ainsi à :

$$
P(it) = \frac{3}{26} \approx 0.1154
$$

In [ ]:
p_it = bigrams['it'] / (N - 1)

Dans un modèle *n*-grammes toutefois, la question n’est pas réellement d’estimer la probabilité d’un événement mais plutôt sa vraisemblance en fonction d’un historique. Reformulons : quelle serait la probabilité de l’événement *t* sachant que *i* est arrivé juste avant ? Du point de vue mathématique, la formule revient à :

$$
P(t|i) = \frac{F(it)}{F(i)} = \frac{3}{3} = 1
$$

En effet, dans le corpus, un *i* est toujours suivi d’un *t*. Vérifions avec Python :

In [ ]:
p_t_i = bigrams['it'] / unigrams['i']

Quelle est la lettre la plus probable sachant *e* ?

In [ ]:
alphabet = unigrams.keys()
probabilities = dict()
for c in alphabet:
    key = f"e{c}"
    probabilities.update({
        key: bigrams[key] / unigrams['e']
    })

## Des contraintes pesant sur les modèles

### Formule des probabilités composées

L’intuition première pour calculer des probabilités sur des chaînes de caractères serait d’appliquer une chaîne de traitement sans historicité :

$$
P(u_1 u_2 \ldots u_n) \approx \prod_i P(u_i)
$$

Pour obtenir le mot *chai*, il suffirait ainsi de multiplier entre elles les probabilités de chacune des lettres :

$$
P(chai) = P(c) \cdot P(h) \cdot P(a) \cdot P(i) \approx 6.5649 \times 10^{-5}
$$

In [ ]:
# in a unigram model, what are the odds to get the word 'chai'?
word = "chai"
p_chai = 1
for c in word:
    p_chai *= unigrams[c] / N
p_chai

Et dans un modèle bigramme, la logique reste la même, chaque unité étant cette fois-ci composé de deux caractères :

$$
P(chai) = P(ch) \cdot P(ai) = 0.0016
$$

In [ ]:
# and in a bigram model?
p_chai = 1
for i in range(0, len(word), 2):
    bigram = word[i:i+2]
    p_chai *= bigrams[bigram] / (N - 1)
p_chai

Ce genre de modèle qui ne tient pas compte du contexte reste très limité : un générateur basé sur une telle règle écrirait des caractères aléatoirement. Après tout, les chaînes *du* et *hb* bénéficient de la même estimation de probabilité :

$$
P(du) = P(d) * P(u) \approx 0.0384 \cdot 0.0384 \approx 0.001479\\
P(hb) = P(h) * P(b) \approx 0.0384 \cdot 0.0384 \approx 0.001479
$$

Une meilleure approche consiste à poser le problème sous forme de probabilités conditionnelles :

$$
P(chai) = P(h|c) \cdot P(a|ch) \cdot P(i|cha)
$$

Une formule qui peut se généraliser :

$$
P(w_1 w_2 \ldots w_n) = \prod_{i=1} P(w_i ∣ w_0 \ldots w_{i−1})
$$

Si elle semble prometteuse, sur l’argument qu’elle repose toujours sur l’antériorité, elle aboutit très rapidement à une probabilité de 0 pour des énoncés pourtant probables :

$$
\begin{align}
P(chai) &= P(h|c) \cdot P(a|ch) \cdot P(i|cha)\\
&= \frac{F(ch)}{F(c)} \cdot \frac{F(cha)}{F(ch)} \cdot \frac{F(chai)}{F(cha)}\\
&= \frac{1}{1} \cdot \frac{1}{1} \cdot \frac{0}{1}\\
&= 1 \cdot 1 \cdot 0\\
&= 0
\end{align}
$$

### Hypothèses de correction

Pour éviter cet écueil, la première intuition serait d’ignorer tout simplement les cas qui n’apparaissent pas dans le corpus d’entraînement. Si nous retenons cette idée, $P(chai)$ devient subitement égale à 1. Passer de 0 à 100 % de chances de voir le mot *chai* (autant que le mot *chat*) semble indiquer que notre modèle de langage n’est pas très rationnel.

Une correction plus envisageable, et qui prend le nom de **lissage de Laplace**, serait d’augmenter de 1 toutes les fréquences du corpus et d’assigner la même valeur aux évènements nuls :

$$
\begin{align}
P(chat) &= \frac{F(ch)+1}{F(c)+1} \cdot \frac{F(cha)+1}{F(ch)+1} \cdot \frac{F(chat)+1}{F(cha)+1}\\
&= \frac{2}{2} \cdot \frac{2}{2} \cdot \frac{2}{2}\\
&= 1\\
P(chai) &= \frac{F(ch)+1}{F(c)+1} \cdot \frac{F(cha)+1}{F(ch)+1} \cdot \frac{F(chai)+1}{F(cha)+1}\\
&= \frac{2}{2} \cdot \frac{2}{2} \cdot \frac{1}{2}\\
&= 0.5
\end{align}
$$

En poursuivant cette logique, une meilleure approximation de la probabilité d’un évènement inconnu s’appuierait sur une pondération du maximum de vraisemblance. Fixons le paramètre $\alpha$ à 0,05 pour la probabilité d’un évènement inconnu et un paramètre $k$ pour la pondération qui vaut $1 - \alpha$ soit 0,95 :

$$
P(chai) = 0.95 \times 0 + \frac{0.05}{27} \approx 0.0019
$$

La formule vaut ainsi :

$$
P(w_i) = k \cdot P_{ML}(w_i) + \frac{\alpha}{N}
$$

Dans la réalité, le paramètre $k$ est ajusté sur le corpus d’entraînement plutôt que déterminé arbitrairement. D’autres techniques existent, comme les actualisations de Good-Turing et de Witten-Bell, ou encore le lissage de Kneser-Ney qui estiment les évènements inconnus sur la base des hapax.

### L’hypothèse markovienne

En théorie, la formule des probabilités composées couvre bien les besoins impliqués par la construction d’un modèle de langage simple. Après tout, quand un corpus d’apprentissage n’est constitué que de quelques caractères et que le nombre de caractères d’un mot est fini, il semble envisageable de calculer toutes les possibilités.

Étendons notre exemple au français, qui dispose d’un alphabet de 26 lettres et dont le mot le plus long, *anticonstitutionnellement*, en compte 25. Là encore, générer un mot de *n* caractères en partant d’un état initial et en se fondant sur des données d’entraînement suffisamment importantes semble à portée de main. Oui, mais… qu’en est-il des mots composés ? Et dans les autres langues ? Les règles de composition de l’allemand font par exemple le bonheur des écoliers qui inventent les péripéties du capitaine naviguant sur le Danube et le Rhin (*Donaurheinschifffahrtskapitän*) afin d’égaler la prouesse établie par la *Donau­dampf­schiffahrts­elektrizitäten­haupt­betriebs­werk­bau­unter­beamten­gesellschaft*.

Rappelons aussi que l’unité de base des modèles de langage n’est pas le caractère mais le mot. L’alphabet devient lexique et la longueur d’une phrase étant virtuellement infinie, les besoins en calcul sont si délirants qu’il ne sera jamais possible d’estimer de telles probabilités.

Comment faire alors ? C’est ici qu’intervient l’hypothèse markovienne, appelée ainsi après le mathématicien russe Andreï Markov, qui suggère que la probabilité conditionnelle d’un mot sachant son historique est approximée par la probabilité de ce mot sachant uniquement celui qui le précède. Par exemple, la probabilité que le mot *vagues* termine la phrase *Le soleil brille au-dessus des* est proche de celle que le mot *vagues* suive directement l’article *des* :

$$
P(\text{flots}|\text{Le soleil brille au-dessus des}) \approx P(\text{flots}|\text{des})
$$

On parle aussi d’**horizon limité**, une approximation qui peut s’écrire sous la forme :

$$
P(w_{n+1}|w_1 \ldots w_n) \approx P(w_{n+1}|w_n)
$$

Mieux, l’hypothèse markovienne dans sa généralisation définit une fenêtre contextuelle à gauche :

$$
P(w_1 w_2 \ldots w_n) \approx \prod_i P(w_i|w_{i-k} \ldots w_{i-1})
$$

Les modèles probabilistes peuvent s’étendre ainsi aux trigrammes, aux tétragrammes, aux… sauf que le langage fait montre de tant de diversité que les modèles *n*-grammes ne suffisent pas. Prenons un énoncé :

> Le film, encensé par la critique de nombreux médias spécialisés, comme *Les inrockuptibles*, *Télérama*, *ÉcranLarge* ou encore *Première*, qui s’était déjà exprimée favorablement sur l’opus précédent, est fort mauvais.

La structure des phrases, notamment à l’écrit, implique souvent une dépendance longue distance (*long distance dependancy* ou LDD) facilitée par des incises, des relatives ou des tournures interrogatives.